LetsGrowMore

Advanced Level Task 2: Next Word Prediction

Done By: Yash Rai

In [1]:
# Importing Libraries
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
# Loading the Dataset
path = '1661-0.txt'
text = open(path).read().lower()
print('Corpus Length:', len(text))

Corpus Length: 581888


In [3]:
# Splitting Dataset into one word each
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [4]:
# Making list of Sorted Unique Words
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [5]:
# Feature Engineering
WORD_LENGTH = 4
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the']
adventures


In [6]:
# Storing Features and Corresponding Labels
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [7]:
print(X[0][0])

[False False False ... False False False]


In [8]:
# Building the Model
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [9]:
# Training Our Model
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
811/811 [==============================] - 220s 268ms/step - loss: 6.0084 - accuracy: 0.1083 - val_loss: 7.0213 - val_accuracy: 0.0989
Epoch 2/20
811/811 [==============================] - 216s 267ms/step - loss: 5.7588 - accuracy: 0.1482 - val_loss: 7.8365 - val_accuracy: 0.1098
Epoch 3/20
811/811 [==============================] - 217s 268ms/step - loss: 5.7375 - accuracy: 0.1766 - val_loss: 8.0783 - val_accuracy: 0.1023
Epoch 4/20
811/811 [==============================] - 216s 267ms/step - loss: 5.4483 - accuracy: 0.2092 - val_loss: 8.3023 - val_accuracy: 0.1080
Epoch 5/20
811/811 [==============================] - 217s 267ms/step - loss: 5.1646 - accuracy: 0.2478 - val_loss: 8.2617 - val_accuracy: 0.1009
Epoch 6/20
811/811 [==============================] - 217s 268ms/step - loss: 4.9234 - accuracy: 0.2861 - val_loss: 8.5635 - val_accuracy: 0.0934
Epoch 7/20
811/811 [==============================] - 216s 267ms/step - loss: 4.7245 - accuracy: 0.3225 - val_loss: 8.5912 -

In [10]:
# Loading Our Model
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [11]:
# Making Predictions
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [12]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [13]:
# Function for Prediction
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [14]:
s =  "Will you be my partner?"
print("Sentence: ",s)
seq = " ".join(tokenizer.tokenize(s.lower())[0:2])
print("Sequence: ",seq)
print("Next Possible Words: ", predict_completions(seq, 2))

Sentence:  Will you be my partner?
Sequence:  will you
will
you
Next Possible Words:  ['very', 'night']


In [17]:
sen =  "This is not going anywhere."
print("Sentence: ",sen)
sequ = " ".join(tokenizer.tokenize(sen.lower())[0:4])
print("Sequence: ",sequ)
print("Next Possible Words: ", predict_completions(sequ, 4))

Sentence:  This is not going anywhere.
Sequence:  this is not going
this
is
not
going
Next Possible Words:  ['to', 'on', 'for', 'out']
